In [34]:
import pandas as pd
import numpy as np

In [4]:
stock_name = "FB"
training_data = pd.read_csv(stock_name + "/"+stock_name +"_training_set.csv",low_memory=False)

In [5]:
cols = training_data.columns

In [6]:
print cols

Index([u'Unnamed: 0', u'Date', u'Mod Score', u'Average Score', u'Stock Price'], dtype='object')


In [43]:

x = training_data[cols[2:4]] #taking Date and Averagee sentiment
print x

    Mod Score  Average Score
0         1.0            1.0
1         1.0            2.0
2         1.0            1.0
3         1.0            1.0
4         1.0            1.0
5         1.0            2.0
6         1.0            1.0
7         1.0            2.0
8         1.0            2.0
9         1.0            1.0
10        1.0            3.0
11        1.0            2.0
12        2.0            3.0
13        1.0            3.0
14        1.0            1.0
15        1.0            1.0
16        1.0            2.0
17        2.0            3.0
18        1.0            2.0
19        2.0            3.0
20        1.0            1.0
21       -1.0            1.0
22        1.0            1.0
23        1.0            2.0
24        1.0            1.0
25        1.0            2.0
26        1.0            1.0
27        1.0            2.0
28        1.0           -2.0
29        1.0            2.0
30        1.0           -2.0


In [52]:
y = np.asarray(training_data[cols[len(cols)-1]],dtype="|S6")#output
print y

['0.57' '1.27' '0.89' '0.89' '0.39' '0.3' '1.73' '1.15' '0.24' '0.71'
 '1.71' '1.47' '0.18' '1.72' '1.23' '0.95' '0.22' '1.46' '0.01' '0.71'
 '0.65' '0.87' '0.23' '1.19' '2.85' '0.24' '0.85' '0.88' '1.45' '0.3'
 '0.76']


In [54]:
from sklearn import svm
clf = svm.SVC(gamma=0.001, C=100)


In [57]:
values,target = x[:-10],y[:-10]

In [58]:
clf.fit(x,y)

SVC(C=100, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape=None, degree=3, gamma=0.001, kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)

In [61]:
model = x.ix[len(x)-1]
print model
print clf.predict(model)

Mod Score        1.0
Average Score   -2.0
Name: 30, dtype: float64
['1.45']


/Library/Frameworks/Python.framework/Versions/2.7/lib/python2.7/site-packages/sklearn/utils/validation.py:395: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  DeprecationWarning)


### 